In [1]:
#Brain_Segmentation
import os, sys, glob
import SimpleITK as sitk
import sys
import os
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn.utils import shuffle
%matplotlib inline

from skimage.util.dtype import dtype_range
from skimage.util import img_as_ubyte
from skimage import exposure
from skimage.morphology import disk
from skimage.filters import rank
from scipy.ndimage import gaussian_filter
from skimage import data
from skimage import img_as_float
from skimage.morphology import reconstruction
from scipy import ndimage
import random
from random import randrange
from keras.utils import np_utils

from importlib import reload  # Python 3.4+ only.
import visualization_utils  as vu
from visualization_utils import show_lable_on_image4
reload(vu)

import os, sys, glob
import numpy as np
import SimpleITK as sitk
import sys
import os
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn.utils import shuffle
%matplotlib inline

from skimage.util.dtype import dtype_range
from skimage.util import img_as_ubyte
from skimage import exposure
from skimage.morphology import disk
from skimage.filters import rank
from scipy.ndimage import gaussian_filter
from skimage import data
from skimage import img_as_float
from skimage.morphology import reconstruction
from scipy import ndimage
import random

import model_unet
reload(model_unet)

IMG_HEIGHT = 240
IMG_WIDTH = 240
IMG_CHANNELS = 4

label_type_shrt = ['background', 'necrotic',
             'edema', 'enhancing']
label_type = ['background', 'necrotic and non-enhancing tumor', 'edema', 'enhancing tumor']

DATA = 'C:/Users/student/Desktop/Naomi_dataset/MICCAI_BraTS_2019_Data_Training/'
VALIDATION_DATA = 'C:/Users/student/Desktop/Naomi_dataset/MICCAI_BraTS_2019_Data_Validation/'
DATA_HGG = DATA +'/HGG/'
DATA_LGG = DATA +'/LGG/'
NUMPY_DIR = 'C:/Users/student/Desktop/Naomi_dataset/train_numpy_images/'
VALIDATION_NUMPY_DIR  = 'C:/Users/student/Desktop/Naomi_dataset/val_numpy_images/'

'''
DATA= 'dicom/MICCAI_BraTS_2019_Data_Training/'
VALIDATION_DATA = 'dicom/MICCAI_BraTS_2019_Data_Validation/'
DATA_HGG = DATA +'/HGG/'
DATA_LGG = DATA +'/LGG/'

NUMPY_DIR = 'dicom/MICCAI_BraTS_2019_Data_Training/numpy_images/'
VALIDATION_NUMPY_DIR = 'dicom/MICCAI_BraTS_2019_Data_Validation/numpy_images/'
'''
FLAIR = 'flair'
T1 = 't1'
T2 = 't2'
T1CE = 't1ce'

img_type=['FLAIR', 'T1','T1CE', 'T2']

model = model_unet.unet(IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS)
model.load_weights('model_unet_4ch.hdf5')

df_train= pd.read_csv('df_train.csv')
df_test= pd.read_csv('df_test.csv')
df_val= pd.read_csv('df_val.csv')

TEST_PRED_NUMPY_DIR = 'C:/Users/student/Desktop/Naomi_dataset/predictions/pred1/test/numpy_images/'
VALIDATION_PRED_NUMPY_DIR = 'C:/Users/student/Desktop/Naomi_dataset/predictions/pred1/validation/numpy_images/'
VALIDATION_PRED_NII_DIR = 'C:/Users/student/Desktop/Naomi_dataset/predictions/pred1/validation/nii/'



def show_pred_im(im, pred):
    
    vu.show_n_images([im[:,:,1], 
                   im[:,:,0],pred,
                  show_lable_on_image4(im[:,:,1], pred[:,:])],
                 titles=['Flair','T1', 'Pred',  'Prediction on Flair'])

def get_numpy_img_lbl(img_id = 'BraTS19_TCIA10_449_1', np_dir=NUMPY_DIR):
    img=np.load(os.path.join(np_dir, img_id+'.npy'))
    lbl=np.load(os.path.join(np_dir, img_id+'_lbl.npy'))
    return img,lbl

def get_img_for_label(lab=2, axis=0, df=df_train,np_dir = NUMPY_DIR):
    
    img_id= random.choice(df[df['lab'+str(lab)] > 0].id.values)
    
    img,lbl = get_numpy_img_lbl(img_id, np_dir)
    ind = np.where(lbl==lab)
    k = random.randrange(len(ind[0]))
    
    if (axis==0):        
        return img[ind[0][k],:,:] , lbl[ind[0][k],:,:]
        
    lb = np.zeros((240,240),dtype=np.int)
    im = np.zeros((240,240,4),dtype=np.float32)
    
    if (axis==1):
        im[40:40+155,:,:]=img[:, ind[1][k],:,:]
        lb[40:40+155,:]=lbl[:, ind[1][k],:]
        return im,lb
    
    if (axis == 2):
        im[40:40+155,:,:]=img[:, :, ind[2][k],:]
        lb[40:40+155,:]=lbl[:,:,ind[2][k]]
        return im,lb
    return None

def lbl_from_cat(cat_lbl):
    
    lbl=0
    if (len(cat_lbl.shape)==3):
        for i in range(1,4):
            lbl = lbl + cat_lbl[:,:,i]*i
    elif (len(cat_lbl.shape)==4):
        for i in range(1,4):
            lbl = lbl + cat_lbl[:,:,:,i]*i
    else:
        print('Error in lbl_from_cat', cat_lbl.shape)
        return None
    return lbl

def normalize_3D_image(img):
    for z in range(img.shape[0]):
        for k in range(4):
            if (img[z,:,:,k].max()>0):
                img[z,:,:,k] /= img[z,:,:,k].max()
    return img

def get_pred(img, threshold=0.5):
    out_img=img.copy()
    out_img=np.where(out_img>threshold, 1,0)
    return out_img

def prediction_from_probabily_3D(img):
    
    int_image = get_pred(img)
    return lbl_from_cat(int_image)

def predict_3D_img_prob(np_file):
    
    np_img = np.load(np_file)
    for_pred_img = np.zeros((155, 240, 240, 4), np.float32)

    # Normalize image
    for_pred_img = normalize_3D_image(np_img)

    mdl_pred_img =  model.predict(for_pred_img)

    #pred_label = prediction_from_probabily_3D(mdl_pred_img)

    return mdl_pred_img




Using TensorFlow backend.


In [2]:
df_val = pd.DataFrame(columns=['id','lab0','lab1','lab2','lab3'])

In [3]:
img_type=['FLAIR', 'T1','T1CE', 'T2']

In [4]:
from importlib import reload  # Python 3.4+ only.
import visualization_utils  as vu
reload(vu)

<module 'visualization_utils' from 'C:\\Users\\student\\Desktop\\BTS\\visualization_utils.py'>

In [5]:
def show_img_lable(img, lbl, modality = 0):
    
    if (len(lbl.shape)> 2):
        lbl[0,0,3]=1 # for uniqe colors in plot
        lbl = lbl_from_cat(lbl)
    vu.show_n_images([img[:,:,modality],lbl, show_lable_on_image4(img[:,:,modality],lbl)],
                    titles = [img_type[modality], 'Label', 'Label on '+ img_type[modality]]);
    

In [6]:
def read_img_sitk(img):
    inputImage = sitk.ReadImage( img )
    inputImage = sitk.Cast( inputImage, sitk.sitkFloat32 )
    image = sitk.GetArrayFromImage(inputImage)
    return image

In [7]:
df = pd.DataFrame(columns=['Grade','id','lab0','lab1','lab2','lab3',
                           'rmin','rmax','cmin','cmax','zmin','zmax'])

In [8]:
def fill_df_from_path(f):
    s = os.path.splitext(f)[0]
    print(s)

    nimg = os.path.join("./pred_ing/"+s+'.npy')
    out_img = os.path.join("./pred_ing/"+s+'_pred.npy')
    
    a = np.save(nimg, f)
    b = np.save(out_img, f)
    
    df = [a, b]
    
    return 

In [ ]:
#img_id='BraTS19_2013_1_1'
#im = np.load(nimg)
#pred_stats = predict_3D_img_prob(nimg)
#pred = prediction_from_probabily_3D(pred_stats)
#show_pred_im(im[88], pred[88])

#Pyqt
from PyQt5.QtWidgets import *
from PyQt5 import uic
from PyQt5 import QtCore, QtGui, QtWidgets
import cv2
import time
import numpy as np
import datetime
import sys
from os import path
from flask import Flask
from collections import deque
from flask import Flask
from flask_ngrok import run_with_ngrok
from PyQt5.QtGui import QPixmap 
from PyQt5.QtCore import QDir, Qt, QPointF, QRegExp, QStringListModel
from PyQt5.QtGui import QImage, QPainter, QPalette, QPixmap, QTransform
from PyQt5.QtWidgets import (QAction, QApplication, QFileDialog, QLabel,
        QMainWindow, QMenu, QMessageBox, QScrollArea, QSizePolicy)
from PyQt5.QtPrintSupport import QPrintDialog, QPrinter
import matplotlib.pyplot as plt
from PIL import Image

#!/usr/bin/python3
import os
import sys

import nibabel

from PyQt5.QtWidgets import QApplication, QFileDialog, QGraphicsScene, QGraphicsView, QLabel, QListView, QMainWindow, \
    QSlider

from CursorGraphicsView import CursorGraphicsView
import numpy

app = Flask(__name__)
#run_with_ngrok(app)

@app.route("/", methods=['POST', 'GET'])

#웹페이지 템플릿
def home(): 
    return '''
            <html lang="en">
            <head>
                <meta charset="UTF-8">
                <title>Brain_Tumor_Segmentation</title>

            </head>
            <body>
            <style>
            H1 {
              position: absolute;
              left: 245px;
              top: 100px;
              color:black;
            }
            H2 {
              position: absolute;
              left: 250px;
              top: 200px;
              color:black;
            }
            H3 {
              position: absolute;
              left: 420px;
              top: 650px;
              color:black;
            }
            </style>
            <H1>Brain Tumor Segmentation</H1>
            <H2>
            <a href='https://ifh.cc/v-esZ9uw' target='_blank'><img src='https://ifh.cc/g/esZ9uw.png' border='0'></a>
            </H2>

            <H3><form action="/BTS/" method="post"><button name="forwardBtn" type="submit">BTS.exe</button></H3></form>
            </body>
            </html>
            '''

@app.route("/BTS/", methods=['POST', 'GET'])

#메인함수
def BTS():
    #UI파일 연결
    #단, UI파일은 Python 코드 파일과 같은 디렉토리에 위치해야한다.

    #화면을 띄우는데 사용되는 Class 선언
    #!/usr/bin/env python

    class MainWindow(QMainWindow):
        def __init__(self):
            super().__init__()
            uic.loadUi('mainwindow.ui', self)

            self.action_Open.triggered.connect(self.open_file)

            self.image_labels = self.findChildren(QLabel, QRegExp("image_slice_label_."))
            self.image_sliders = self.findChildren(QSlider, QRegExp("image_slider_."))
            self.image_viewers = self.findChildren(QGraphicsView, QRegExp("image_viewer_."))
            self.contrast_sliders = self.findChildren(QSlider, QRegExp("image_contrast_.*_slider"))
            self.points_list = self.findChild(QListView, 'points_list')
            self.points_model = QStringListModel()
            self.points = []

            self.points_list.setModel(self.points_model)
            self.setWindowTitle('Nifti viewer')
            self.show()

            # redraw viewers whenever a slider is moved
            for i in range(3):
                self.image_sliders[i].valueChanged.connect(lambda value, i=i: self.draw_viewer(i))

            # redraw all viewers whenever a contrast slider is moved
            for slider in self.contrast_sliders:
                slider.valueChanged.connect(self.draw_viewers)

            self.image_cycle_slider.valueChanged.connect(self.cycle_images)

        def open_file(self):
            file_info = QFileDialog.getOpenFileName(parent=self, directory=os.path.expanduser('~'), filter='*.nii *.nii.gz')
            
            ####여기####
            #npy로 변환
            nimg = fill_df_from_path(file_info)
            im = np.load(nimg)
            #학습된 결과를 받아, 예측
            #pred_stats = predict_3D_img_prob(nimg)
            #pred = prediction_from_probabily_3D(pred_stats)
            #show_pred_im(im[88], pred[88])
            
            #for index, row in df_val.iterrows():

            #img_id = row['id']

            #nimg = os.path.join(VALIDATION_NUMPY_DIR, img_id+'.npy')
            npy = nimg[0]
            pred_stats = predict_3D_img_prob(npy)

            pred = prediction_from_probabily_3D(pred_stats)

            #out_img = os.path.join(VALIDATION_PRED_NUMPY_DIR, img_id+'_pred.npy')
            pred_npy = nimg[1]
            np.save(pred_npy, pred)

            pred = np.where(pred==3,4, pred)
            out_nii = os.path.join(VALIDATION_PRED_NII_DIR, img_id+'.nii.gz')

            sitk_img = sitk.GetImageFromArray(pred)
            sitk.WriteImage(sitk_img , out_nii)
            #######################################3    
            file_info = pred
            
            #pred_stats = predict_3D_img_prob(nimg)
            #pred = prediction_from_probabily_3D(pred_stats)
            
            ####
            if not os.path.isfile(file_info[0]): return

            # menu buttons for saving and deleting points are enabled only once an image has been loaded
            self.action_Save.triggered.connect(self.save_point)
            self.action_Save_points_to_file.triggered.connect(self.save_points_to_file)
            self.action_Delete.triggered.connect(self.delete_point)

            for i, save_button in enumerate([self.action_Save_sagittal_slice,
                                             self.action_Save_coronal_slice,
                                             self.action_Save_transverse_slice]):
                save_button.triggered.connect(lambda value, i=i: self.save_slice(i))

            self.niba_img = nibabel.load(file_info[0])
            data = self.niba_img.get_data()
            self.image_min = data.min()
            self.image_max = data.max()
            self.num_image = 0

            # if the image is 4D, prepare the 4th image slider
            if len(data.shape) == 4:
                self.image_cycle_slider.setMaximum(data.shape[3] - 1)

            # data.shape contains the number of slices of each dimension, which are the maximums of each slider
            for i, slider in enumerate(self.image_sliders):
                slider.setMaximum(data.shape[i] - 1)
                slider.setValue(0)

            # contrast sliders should have a range going from the minimum value of the images to their maximum
            for slider in self.contrast_sliders:
                slider.setMinimum(self.image_min)
                slider.setMaximum(self.image_max)

            self.contrast_sliders[0].setValue(self.image_min)
            self.contrast_sliders[1].setValue(self.image_max)

            # the custom viewers need some data to work properly, and need to draw the slices a first time once the image
            # has been loaded
            for i, viewer in enumerate(self.image_viewers):
                viewer.set_num(i)
                viewer.set_viewers(self.image_viewers)
                viewer.set_sliders(self.image_sliders)
                self.draw_viewer(i)

        def save_point(self):
            if CursorGraphicsView.coords is None: return
            self.points.append(str(CursorGraphicsView.coords + [self.num_image or 0]))
            self.points_model.setStringList(self.points)

        def save_points_to_file(self):
            file_info = QFileDialog.getSaveFileName(parent=self, directory=os.path.expanduser('~'), filter='*.txt')

            try:
                file = open(file_info[0], 'w')
                file.write(os.linesep.join(map(str, self.points)))
            except:
                pass

        def delete_point(self):
            for selection in self.points_list.selectedIndexes():
                self.points.pop(selection.row())

            self.points_model.setStringList(self.points)

        def save_slice(self, num: int):
            file_info = QFileDialog.getSaveFileName(parent=self, directory=os.path.expanduser('~'), filter='*.png')

            if file_info[0] == '': return

            # python-pillow can load any kind of image and save it in any common format
            image = Image.fromqpixmap(self.image_viewers[num].scene().items()[-1].pixmap())
            image.save(file_info[0], 'PNG')

        def draw_viewer(self, num_slider: int):
            if self.niba_img is None: return

            data = self.niba_img.get_data()
            header = self.niba_img.get_header()

            if data is None or header is None: return

            self.image_labels[num_slider].setText(self.tr("Slice " + str(self.image_sliders[num_slider].value())))

            # we need a 2D array from a 3D or 4D array to display as an image

            # the slice(None) index will take an entire dimension, so using 2 of them and a number will reduce the
            # dimensions of the original array by one if the image is 3D
            slice_range = [slice(None)] * 3
            slice_range[num_slider] = self.image_sliders[num_slider].value()
            slice_range = tuple(slice_range)

            # if the original image is 4D, we need to further reduce the number of dimensions by selecting a 3D image
            if len(data.shape) == 4: slice_range = slice_range + (self.num_image,)

            # after the 2D plane has been extracted, the contrast must be changed according to the contrast sliders' values
            plane = numpy.copy(data[slice_range])
            image_min = self.image_contrast_min_slider.value()
            image_max = self.image_contrast_max_slider.value()
            plane[plane < image_min] = image_min
            plane[plane > image_max] = image_max

            if self.image_min == self.image_max: self.image_max = 1

            # the plane needs to be converted to 256-level greyscale image
            converted = numpy.require(numpy.divide(numpy.subtract(plane, image_min), (image_max - image_min) / 255),
                                      numpy.uint8, 'C')
            # the plane next needs to be scaled according to the scales in the NIfTI header
            transform = QTransform()
            scales_indexes = [((x + num_slider) % 3) + 1 for x in [1, 2]]
            scale = QPointF(header['pixdim'][min(scales_indexes)], header['pixdim'][max(scales_indexes)])

            # the plane needs to be rotated in order to be properly displayed
            transform.scale(scale.x(), scale.y())
            transform.rotate(-90)

            pixmap = Image.fromarray(converted).toqpixmap().transformed(transform)
            scene = QGraphicsScene(0, 0, pixmap.width(), pixmap.height())

            scene.addPixmap(pixmap)
            self.image_viewers[num_slider].setScene(scene)
            self.image_viewers[num_slider].set_scale(scale)
            self.image_viewers[num_slider].make_cursor()

        def draw_viewers(self):
            for i in range(3): self.draw_viewer(i)

        def cycle_images(self, value: int):
            self.num_image = value
            self.draw_viewers()


    if __name__ == '__main__':
        app = QApplication(sys.argv)
        MainWindow()
        sys.exit(app.exec())
        
    return home()
        
if __name__ == "__main__" :
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Jun/2020 09:16:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2020 09:16:45] "GET / HTTP/1.1" 200 -


TypeError: expected str, bytes or os.PathLike object, not tuple

AttributeError: 'MainWindow' object has no attribute 'niba_img'

AttributeError: 'MainWindow' object has no attribute 'niba_img'

...

AttributeError: 'MainWindow' object has no attribute 'niba_img'

AttributeError: 'MainWindow' object has no attribute 'niba_img'

AttributeError: 'MainWindow' object has no attribute 'niba_img'

AttributeError: 'MainWindow' object has no attribute 'niba_img'

AttributeError: 'MainWindow' object has no attribute 'niba_img'